In [3]:
# 导入所需的库
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import AmlCompute, Environment, Model,  ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

In [ ]:
# 创建 Azure ML 客户端
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="4384af70-f27b-4b56-923b-90b75e7d7151",
    resource_group_name="RG001",
    workspace_name="dishclassifier"
)

In [ ]:
# # 注册数据集
# data = Data(name="my_dataset", path="path/to/data")
# ml_client.data.create_or_update(data)

In [ ]:
# 定义模型路径
model_path = "best_model.keras"
model = Model(
    path=model_path,
    name="best_model",
    description="Dish Classifer",
    type=AssetTypes.CUSTOM_MODEL
)

In [ ]:
# 上传模型到工作区
registered_model = ml_client.models.create_or_update(model)


In [ ]:
# 配置环境
env = Environment(
    name="inference_env",
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest",
    conda_file="path_to_your_conda_dependencies_file"
)

In [ ]:
# 创建端点
endpoint_name = "image-classification-endpoint"
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="identify 10 Singaporean dishes"
)
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# 创建并部署在线推理服务
deployment = ManagedOnlineDeployment(
    name="best-model-service13",
    endpoint_name=endpoint_name,
    model=registered_model,
    environment=env,
    instance_type="Standard_DS3_v2",
    instance_count=1,
    entry_script="score.py"
)
ml_client.online_deployments.begin_create_or_update(deployment).result()

print(f"Endpoint {endpoint_name} is ready.")